In [ ]:
# import numpy as np
# import pyaudio
# import time
# import librosa

# class AudioHandler(object):
#     def __init__(self):
#         self.FORMAT = pyaudio.paFloat32
#         self.CHANNELS = 1
#         self.RATE = 44100
#         self.CHUNK = 1024 * 2
#         self.p = None
#         self.stream = None

#     def start(self):
#         self.p = pyaudio.PyAudio()
#         self.stream = self.p.open(format=self.FORMAT,
#                                   channels=self.CHANNELS,
#                                   rate=self.RATE,
#                                   input=True,
#                                   output=False,
#                                   stream_callback=self.callback,
#                                   frames_per_buffer=self.CHUNK)

#     def stop(self):
#         self.stream.close()
#         self.p.terminate()

#     def callback(self, in_data, frame_count, time_info, flag):
#         numpy_array = np.frombuffer(in_data, dtype=np.float32)
#         librosa.feature.mfcc(numpy_array)
#         return None, pyaudio.paContinue

#     def mainloop(self):
#         while (self.stream.is_active()): # if using button you can set self.stream to 0 (self.stream = 0), otherwise you can use a stop condition
#             time.sleep(2.0)


# audio = AudioHandler()
# audio.start()     # open the the stream
# audio.mainloop()  # main operations with librosa
# audio.stop()

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

import tensorflow_io as tfio

In [ ]:
# From tutorial:

'''
https://www.youtube.com/watch?v=AShHJdSIxkY
'''

import pyaudio 
import struct
import numpy as np

import matplotlib.pyplot as plt

%matplotlib tk

import pyaudio
p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
        if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))

CHUNK = 1024 * 4
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100


p = pyaudio.PyAudio()

stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)


data = stream.read(CHUNK)
data




In [ ]:
mel_model = tf.keras.models.load_model('../data/saved_models/mel_1_sec_model_(95acc)/mel_1_sec_model')

# Check its architecture
mel_model.summary()

In [ ]:
def get_stft_snapshots(index, stft_dict):
    
    r_id = index['recording_id']

    f_path = get_filepath(r_id)
    
    audio = tfio.audio.AudioIOTensor(f_path)
    sr = audio.rate

#     dur = get_duration(index)
    dur = 1
    
    for i in range(dur):
        
        start = i + 1 * sr * floor(index['t_min'])
        stop = start + sr

        audio_slice = audio[start: stop + 1]
        
        audio_tensor = tf.squeeze(audio_slice, axis=[-1])
        
        tensor = tf.cast(audio_tensor, tf.float32)
        
        spectrogram = tfio.experimental.audio.spectrogram(
            tensor,
            nfft=512,
            window=512,
            stride=256
        )
        
        mel_spectrogram = tfio.experimental.audio.melscale(
            spectrogram, 
            rate=sr, 
            mels=128, 
            fmin=93.75,     # 20, 
            fmax=11627.90) # 15000)
            
        dbscale_mel_spectrogram = tfio.experimental.audio.dbscale(
            mel_spectrogram, 
            top_db=80)
        
        
        stft_dict['dbscale_mel'].append(mel_spectrogram.numpy())
        stft_dict['mel'].append(mel_spectrogram.numpy())
        stft_dict['stft'].append(spectrogram.numpy())
        stft_dict['species_id'].append(index['species_id'])
    
    
    return None

In [ ]:
def prep_audio(audio_input):
    
    audio = tfio.auido.AudioIOTensor(audio_input)
    sr = audio.rate
    
    audio_tensor = tf.squeeze(audio, axis=[-1])
    
    tensor = tf.cast(audio_tensor, tf.float32)
    
    spectrogram = tfio.experimental.audio.spectrogram(
        tensor,
        nfft=512,
        window=512,
        stride=256)

    mel_spectrogram = tfio.experimental.audio.melscale(
        spectrogram, 
        rate=sr, 
        mels=128, 
        fmin=93.75,     # 20, 
        fmax=11627.90) # 15000)

    dbscale_mel_spectrogram = tfio.experimental.audio.dbscale(
        mel_spectrogram, 
        top_db=80)
    
    
    return dbscale_mel_spectrogram

# Resources:

PyAudio
-[Insatlling PyAudio on Python 3.7](https://stackoverflow.com/questions/54998028/how-do-i-install-pyaudio-on-python-3-7)
- [Select Specific Input Device with PyAudio](https://stackoverflow.com/questions/36894315/how-to-select-a-specific-input-device-with-pyaudio)  


- [Let's Build an Audio Spectrum Analyzer in Python (pt1)](https://www.youtube.com/watch?v=AShHJdSIxkY)  

Librosa
- [Librosa with Microphone Input](https://stackoverflow.com/questions/59056786/python-librosa-with-microphone-input)  
- [Librosa Documentation](https://librosa.org/doc/latest/core.html#signal-generation)


Tensorflow
- [Tensorflow Audio Data Prep and Augmentation](https://www.tensorflow.org/io/tutorials/audio#read_an_audio_file)  
- [Save and load models](https://www.tensorflow.org/tutorials/keras/save_and_load) 
- 




- [Paper: Speech Recognition](https://arxiv.org/pdf/1904.08779.pdf)





In [ ]:
# import sounddevice as sd
# print sd.query_devices() 


pyaudio.Stream(PA_manager, 
               rate, 
               channels, 
               format, 
               input=False, 
               output=False, 
               input_device_index=None, 
               output_device_index=None, 
               frames_per_buffer=1024, 
               start=True, 
               input_host_api_specific_stream_info=None, 
               output_host_api_specific_stream_info=None, 
               stream_callback=None)

In [ ]:
import pyaudio
import wave

chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 1
fs = 44100  # Record at 44100 samples per second
seconds = 3
filename = "output.wav"

p = pyaudio.PyAudio()  # Create an interface to PortAudio

print('Recording')

stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)

frames = []  # Initialize array to store frames

# Store data in chunks for 3 seconds
for i in range(0, int(fs / chunk * seconds)):
    print('stream')
    data = stream.read(chunk)
    frames.append(data)

# Stop and close the stream 
stream.stop_stream()
stream.close()
# Terminate the PortAudio interface
p.terminate()

print('Finished recording')

# Save the recorded data as a WAV file
wf = wave.open(filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()